In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from itertools import product

# Load your dataset (assuming train_data is a Pandas DataFrame)
train_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/train_set.csv")

In [ ]:
train_data.head()

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,adjusted_ride_cost
0,58,39,Suburban,Silver,72,4.06,Evening,Economy,43,239.189243
1,42,31,Rural,Silver,0,3.99,Afternoon,Premium,76,413.308193
2,89,28,Rural,Regular,67,4.31,Afternoon,Premium,134,682.703993
3,59,35,Urban,Silver,83,3.51,Night,Economy,128,530.087537
4,62,39,Rural,Gold,83,3.59,Afternoon,Premium,47,346.764369


In [ ]:
# Selecting numerical variables for outlier detectionA
num_features = ["Number_of_Riders", "Number_of_Drivers", "Number_of_Past_Rides","Average_Ratings","Expected_Ride_Duration","adjusted_ride_cost"]
data = train_data[num_features]

# Define hyperparameter grid
n_estimators_values = [50, 100, 200]   # Number of trees
max_samples_values = [256, 512, 1024]  # Sample size per tree
contamination_values = np.arange(0.01, 0.05, 0.01)  # Outlier proportion

# Store results
best_params = None
best_outlier_count = 0

# Grid search over all parameter combinations
for n_estimators, max_samples, contamination in product(n_estimators_values, max_samples_values, contamination_values):
    # Train Isolation Forest model
    iso_forest = IsolationForest(n_estimators=n_estimators, max_samples=max_samples, contamination=contamination, random_state=42)
    iso_forest.fit(data)

    # Get anomaly predictions (-1 = outlier, 1 = inlier)
    outliers = iso_forest.predict(data)

    # Count number of detected outliers
    total_outliers = np.sum(outliers == -1)

    # Store best parameters based on maximum outliers detected
    if total_outliers > best_outlier_count:
        best_outlier_count = total_outliers
        best_params = {
            "n_estimators": n_estimators,
            "max_samples": max_samples,
            "contamination": contamination
        }

# Print best parameters
print("Best Parameters:", best_params)

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_iforest.py:336: UserWarning: max_samples (1024) is greater than the total number of samples (800). max_samples will be set to n_samples for estimation.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_iforest.py:336: UserWarning: max_samples (1024) is greater than the total number of samples (800). max_samples will be set to n_samples for estimation.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_iforest.py:336: UserWarning: max_samples (1024) is greater than the total number of samples (800). max_samples will be set to n_samples for estimation.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_iforest.py:336: UserWarning: max_samples (1024) is greater than the total number of samples (800). max_samples will be set to n_samples for estimation.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_iforest.py:336: UserWarning: max_samples (1024) is greater than the to

Best Parameters: {'n_estimators': 50, 'max_samples': 256, 'contamination': np.float64(0.04)}


In [ ]:
# Re-train Isolation Forest with best parameters
best_forest = IsolationForest(n_estimators=best_params["n_estimators"],
                              max_samples=best_params["max_samples"],
                              contamination=best_params["contamination"],
                              random_state=42)
best_forest.fit(data)

# Get final anomaly predictions
outlier_labels = best_forest.predict(data)

# Count outliers per variable
outlier_counts = {
    "Number_of_Riders_outliers": np.sum((outlier_labels == -1) & (data["Number_of_Riders"] > data["Number_of_Riders"].quantile(0.99))),
    "Number_of_Drivers_outliers": np.sum((outlier_labels == -1) & (data["Number_of_Drivers"] > data["Number_of_Drivers"].quantile(0.99))),
    "Number_of_Past_Rides_outliers": np.sum((outlier_labels == -1) & (data["Number_of_Past_Rides"] > data["Number_of_Past_Rides"].quantile(0.99))),
    "Average_Ratings_outliers": np.sum((outlier_labels == -1) & (data["Average_Ratings"] > data["Average_Ratings"].quantile(0.99))),
    "Expected_Ride_Duration_outliers": np.sum((outlier_labels == -1) & (data["Expected_Ride_Duration"] > data["Expected_Ride_Duration"].quantile(0.99))),
    "adjusted_ride_cost_outliers": np.sum((outlier_labels == -1) & (data["adjusted_ride_cost"] > data["adjusted_ride_cost"].quantile(0.99)))

}

# Print outlier counts per variable
print("Outlier Counts Per Variable:", outlier_counts)

Outlier Counts Per Variable: {'Number_of_Riders_outliers': np.int64(0), 'Number_of_Drivers_outliers': np.int64(4), 'Number_of_Past_Rides_outliers': np.int64(0), 'Average_Ratings_outliers': np.int64(0), 'Expected_Ride_Duration_outliers': np.int64(2), 'adjusted_ride_cost_outliers': np.int64(5)}
